# Chapter 01 — Python, Linear Algebra, and Numerical Modeling (for FEM)

**Duration:** ~3 hours (with optional extras)  
**Audience:** Students with varying programming/math backgrounds  
**Goal:** Build the minimum Python + linear algebra toolkit and the mental model of *numerical modeling* you need before assembling finite elements.

---
### Table of Contents
1. What libraries we use (and why)
2. Numerical modeling primer (big picture)
3. Linear algebra crash course for FEM
4. Floating-point arithmetic: precision, stability, and pitfalls
5. Python quick primer (scientific computing)
6. Matrix creation & operations in NumPy
7. Indexing, slicing, and broadcasting
8. Dense vs. sparse matrices (and formats)
9. Solving linear systems: direct vs. iterative
10. Boundary conditions and constraints (Dirichlet/Neumann/Penalty)
11. Mini FEM: 1D bar with 2-node elements
12. Exercises (practice)

Use this notebook interactively—run, edit, and experiment. ✨

## 1. What libraries we use (and why)

**NumPy**
- Core library for arrays, vectors, and matrices in Python.
- Provides fast linear algebra (wrapping optimized BLAS/LAPACK).
- We use it to assemble and manipulate vectors/matrices: `@` for matrix–vector products, `np.linalg.solve` for small systems.

**SciPy** (optional but highly recommended)
- Builds on NumPy and adds scientific algorithms (optimization, integration, signal processing).
- For FEM, the key bit is `scipy.sparse`: sparse matrices (`csr`, `coo`) and solvers (`spsolve`, iterative methods).
- Use when your global stiffness matrix is large and sparse.

**Matplotlib**
- Basic plotting—good for quick sanity checks and result visualization.
- We'll use it to plot displacement/stress in simple 1D examples.

**When to use which?**
- **Small demos (< few thousand DoF):** Dense NumPy is fine.
- **Realistic FEM (10⁴–10⁶ DoF):** Use SciPy sparse + iterative solvers; avoid building full dense matrices.

## 2. Numerical modeling primer (big picture)

**What is a model?** A simplified, mathematical description of a physical system. For solid mechanics:
- Governing equations (e.g., equilibrium, compatibility, constitutive law).
- Domain $\Omega$ and boundary $\partial\Omega$ with **boundary conditions** (Dirichlet: displacements; Neumann: tractions).

**From PDE to algebra:**
1. Start with strong form (differential equations).
2. Derive weak/variational form (integrate by parts; natural vs essential BCs emerge).
3. Discretize the domain (mesh) and the solution space (shape functions).
4. Assemble linear system: $K u = f$.

**Three pillars of a good numerical method**
- **Consistency:** Discretization matches the continuous equations (as $h\to0$, truncation error $\to 0$).
- **Stability:** Errors do not blow up during the computation.
- **Convergence:** As the mesh is refined, numerical solution approaches the exact solution.

**Validation vs Verification**
- **Verification:** Are we solving the equations right? (Code correctness; compare to analytical solutions, Method of Manufactured Solutions.)
- **Validation:** Are we solving the right equations? (Compare to experiments/reality.)

**Workflow you’ll repeat in FEM**
1. Define geometry, materials, BCs/loads.
2. Choose discretization (elements, order).
3. Assemble $K$ and $f$.
4. Apply constraints cleanly.
5. Solve, post-process, and check residuals/energy.

## 3. Linear algebra crash course for FEM

**Vectors and inner product**  
Given vectors $v,w\in\mathbb{R}^n$, inner product $\langle v,w\rangle = v^T w$; norm $\|v\|_2=\sqrt{v^T v}$.

**Matrices**  
- Symmetric Positive Definite (SPD): $x^TAx>0$ for $x\ne0$. Stiffness matrices in linear elasticity are SPD (after Dirichlet BCs).
- Condition number $\kappa(A)=\|A\|\|A^{-1}\|$ measures sensitivity: large $\kappa$ means small changes in $b$ can cause big changes in $x$.
- Eigenvalues/eigenvectors: critical for understanding conditioning and convergence of iterative solvers.

**Block matrices (assembly view)**  
Local element matrices scatter-add into the global $K$—a block-sparse pattern emerges.

**Practical tip:** Always inspect residual $r=K u - f$ and report $\|r\|_2$.

In [ ]:
import numpy as np
# Condition number demo on a simple tridiagonal family
def tridiag_dense(n, a=-1.0, b=2.0, c=-1.0):
    A = np.zeros((n,n))
    for i in range(n):
        A[i,i]=b
        if i>0: A[i,i-1]=a
        if i<n-1: A[i,i+1]=c
    return A

for n in [10, 20, 40, 80]:
    A = tridiag_dense(n)
    print(f"n={n:>3}  cond2(A)≈ {np.linalg.cond(A):.2e}")

## 4. Floating-point arithmetic: precision, stability, and pitfalls

- Double precision (IEEE 754) has ~16 decimal digits of precision; rounding happens at each operation.
- **Catastrophic cancellation:** subtracting nearly equal numbers loses significant digits.
- **Ill-conditioning** amplifies rounding errors (large $\kappa$).
- **Scaling:** Nondimensionalize or scale units so entries in $K$ are of comparable magnitude.

**Demo: cancellation**

In [ ]:
import math
x = 1e-8
a = (1 - math.cos(x)) / (x**2)
b = 0.5 * (math.sin(x)/x) * (math.sin(x)/x)  # algebraically equivalent, numerically stable
print('Direct  :', a)
print('Stable  :', b)
print('Abs diff:', abs(a-b))

## 5. Python quick primer (scientific computing)

In [ ]:
# Basic types & containers
x = 3.0; i = 5; v_list = [1,2,3]; t_tuple = (4,5,6)
print(type(x), type(i), type(v_list), type(t_tuple))

import numpy as np
v = np.array([1.,2.,3.]); w = np.array([4.,5.,6.])
print('v+w=', v+w)
print('v·w=', v@w)
print('‖v‖2=', np.linalg.norm(v))

## 6. Matrix creation & operations in NumPy

In [ ]:
A = np.array([[2., -1., 0.], [-1., 2., -1.], [0., -1., 2.]])
b = np.array([1., 0., 1.])
x = np.linalg.solve(A, b)
r = A@x - b
print('x=', x)
print('‖r‖2=', np.linalg.norm(r))
print('Symmetric?', np.allclose(A, A.T))
print('κ(A)=', np.linalg.cond(A))

## 7. Indexing, slicing, and broadcasting

In [ ]:
u = np.arange(10.)
print('u[2:7]=', u[2:7])
print('u[::2]=', u[::2])
mask = (u%2==0)
print('u[mask]=', u[mask])
import numpy as np
M = np.ones((3,4)); row = np.array([0.,1.,2.,3.])
print('M+row=\n', M+row)

## 8. Dense vs. sparse matrices (and formats)

**Why sparse?** Global stiffness matrices are mostly zeros. Storing only nonzeros saves memory and speeds up operations.

**Common formats:**
- **COO (Coordinate):** triplets `(row, col, data)`; easy to build incrementally.
- **CSR (Compressed Sparse Row):** efficient for matrix–vector products and solves; good final format.

**Assembly tip:** accumulate in COO (or Python lists), then convert to CSR once.


In [ ]:
try:
    import scipy.sparse as sp
    import scipy.sparse.linalg as spla
    HAS_SCIPY=True
except Exception:
    HAS_SCIPY=False

def tridiag_sparse(n, a=-1.0, b=2.0, c=-1.0):
    import numpy as np
    import scipy.sparse as sp
    i = np.arange(n)
    data = [a*np.ones(n-1), b*np.ones(n), c*np.ones(n-1)]
    offsets = [-1,0,1]
    return sp.diags(data, offsets, shape=(n,n), format='csr')

if HAS_SCIPY:
    A = tridiag_sparse(10)
    print('CSR nnz=', A.nnz)
else:
    print('SciPy not available — skipping sparse demo')

## 9. Solving linear systems: direct vs. iterative

**Direct (LU/Cholesky):** robust, exact up to rounding, but memory-heavy for large sparse systems (fill-in).

**Iterative (CG, MINRES, GMRES):** use only matrix–vector products; scale to large systems; require **preconditioning** (e.g., ILU, Jacobi) to converge fast.

**Rules of thumb:**
- SPD matrix $\Rightarrow$ Conjugate Gradient (CG) + preconditioner.
- Nonsymmetric $\Rightarrow$ GMRES or BiCGSTAB.


In [ ]:
if HAS_SCIPY:
    import numpy as np
    import scipy.sparse as sp
    import scipy.sparse.linalg as spla
    n = 200
    A = tridiag_sparse(n)
    b = np.ones(n)
    x_cg, info = spla.cg(A, b, tol=1e-10, maxiter=1000)
    print('CG info (0 means success):', info)
    print('Residual:', np.linalg.norm(A@x_cg - b))
else:
    print('Install SciPy to try CG/GMRES demos.')

## 10. Boundary conditions and constraints

**Dirichlet (essential):** prescribed displacements (e.g., $u=0$ at a clamped end).  
**Neumann (natural):** prescribed tractions/forces (e.g., end load).

**Common implementation strategies:**
- **Row/column elimination:** remove constrained DoFs and modify the RHS.
- **Penalty method:** add a large value $\alpha$ on diagonal to enforce $u\approx u_0$ (choose $\alpha$ big but not so big that $\kappa$ explodes).
- **Lagrange multipliers:** enforce constraints exactly by augmenting the system (saddle-point structure).

**Units & scaling:** Keep $E$, $A$, $L$, loads in consistent units to avoid hidden conditioning issues.

## 11. Mini FEM: 1D bar with 2-node linear elements

We solve $-(EA\,u')' = f$ on $[0,L]$ with $u(0)=0$ and traction $t$ at $x=L$.

**Element matrix:** For uniform element length $h$,  
$k_e = \dfrac{EA}{h} \begin{bmatrix}1 & -1\\-1 & 1\end{bmatrix}$.

**Assembly:** Scatter-add $k_e$ into global $K$ using the element connectivity.  
**BCs:** Apply Dirichlet at node 0 via row/column elimination; Neumann enters as a force on the last node.  
**Post-process:** $\varepsilon = u'\approx (u_{i+1}-u_i)/h$, $\sigma=E\,\varepsilon$.

In [ ]:
import numpy as np
def fem_1d_bar(E=210e9, A=1e-4, L=1.0, Ne=10, body=0.0, traction=1000.0):
    nn = Ne + 1
    h  = L / Ne
    K  = np.zeros((nn, nn))
    f  = np.zeros(nn)

    ke = (E*A/h) * np.array([[1., -1.], [-1., 1.]])
    fe_body = (body*h/2.0) * np.array([1., 1.])

    for e in range(Ne):
        n1, n2 = e, e+1
        dofs = [n1, n2]
        K[np.ix_(dofs, dofs)] += ke
        f[dofs] += fe_body

    f[-1] += traction

    # Dirichlet at node 0: u(0)=0
    K = K[1:,1:]
    f = f[1:]

    u_free = np.linalg.solve(K, f)
    u = np.concatenate(([0.0], u_free))
    x = np.linspace(0., L, nn)

    h = L/Ne
    strain = np.diff(u)/h
    stress = E*strain
    x_mid  = x[:-1] + 0.5*h
    return x, u, x_mid, strain, stress

x, u, xm, eps, sig = fem_1d_bar(Ne=20)
print('Max |u|:', np.max(np.abs(u)))
print('Max |σ|:', np.max(np.abs(sig)))

### Plotting

In [ ]:
import matplotlib.pyplot as plt
plt.figure(); plt.plot(x, u, marker='o'); plt.xlabel('x [m]'); plt.ylabel('u [m]'); plt.title('1D bar — displacement'); plt.show()
plt.figure(); plt.plot(xm, sig, marker='s'); plt.xlabel('x [m]'); plt.ylabel('σ [Pa]'); plt.title('1D bar — stress'); plt.show()

## 12. Exercises (practice)

**A. Library warm-up**  
1. Use NumPy to create a random SPD matrix via `A = M.T@M + αI`. Solve `Ax=b` and report `‖Ax-b‖2`.
2. If SciPy is available, build a sparse 1D Poisson matrix with Dirichlet BCs using COO→CSR and solve with `spsolve`.

**B. Linear algebra & conditioning**  
1. Compute `cond(A)` for tridiagonal matrices with sizes 20, 40, 80, 160 and plot the trend.  
2. Build a **Hilbert** matrix (ill-conditioned) for `n=5..12` and observe solution sensitivity.

**C. Floating-point**  
1. Reproduce the cancellation example with different `x` values; compare stable vs. unstable formulas.

**D. Boundary conditions**  
1. Modify the mini FEM to use a penalty method for `u(0)=0` with penalty `α`. Study how `α` affects `κ(K)` and the solution error.

**E. Verification**  
1. For `body=0`, the analytical solution for end traction `t` is `u(x)=t x/(E A)`. Compare numerical `u` at nodes to the exact values.

**F. Stretch goal**  
1. Replace the constant `E` with a spatially varying `E(x)`; assemble element-wise using the element midpoint value.